In [5]:
import pandas as pd
from pymongo import MongoClient
data_dir = '../data/'

In [16]:
#CONFIG DATABASE Connection
user = 'soilspec4gg'
password = 'soilspec4gg'
host = 'api.soilspectroscopy.org'
db_name = 'soilspec4gg'
url = f"mongodb://{user}:{password}@{host}/{db_name}?ssl=true"

client = MongoClient(url, tlsAllowInvalidCertificates=True)
db = client[db_name]

In [45]:
#Some Helper functions
# Mongo is organized in db -> collections (tables) -> records
# A db has collections and each collection has tables


#List collections in db
def list_collections(db):
    return db.list_collection_names()

#query_format = {field: filter_value}
def get_data_collection(db,collection_name,query):
    collection = db[collection_name]

    if query is None:
        query = {}
    
    data = list(collection.find(query))
    return pd.DataFrame(data)

#Get fields in the a specific colletion
def get_collection_fields(db, collection_name):
    collection = db[collection_name]
    # Fetching one document from the collection
    document = collection.find_one()
    if document is not None:
        return document.keys()
    else:
        return []

#Get a sample record from a collection
def get_sample_record(db, collection_name):
    collection = db[collection_name]
    # Fetching one document from the collection
    random_record = collection.aggregate([{'$sample': {'size': 1}}])
    return list(random_record)[0] if random_record else None

# BASIC DATA EXPLORATION

In [33]:
#What collections in data
list_collections()

['soillab', 'mir', 'visnir', 'soilsite']

Available collections:

**soilsite:** site information, e.g., coordinates, pedons, layers, etc.

**soillab_L0:** original soil laboratory data (wet chemistry), e.g., soil texture, carbon, etc.

**soillab_L1:** harmonized soil laboratory data (wet chemistry), e.g., soil texture, carbon, etc.

**mir:** MIR spectral data in the 600-4000 cm-1 range with metadata.

**visnir:** VisNIR spectral data in the 350-2500 nm range with metadata.

**ossl_L0:** Whole OSSL with all separate tables joined at level 0.

**ossl_L1:** Whole OSSL with all separate tables joined at level 1.

In [ ]:
#Open all the data to see main descriptions (Mir and visnir are to big to completely load in my computer)
soillab = get_data_collection(db,'soillab')
# mir = get_data_collection(db,'mir') ##TO BIG TO LOAD
# visnir = get_data_collection(db,'visnir') ##TO BIG TO LOAD
soilsite = get_data_collection(db,'soilsite')

In [36]:
soillab.describe()
#64K records

,id_layer_local_c,acid_tea_usda4b2_cmolkg,al_ox_usda_4g2_wpct,c_tot_usda_4h2_wpct,clay_tot_usda_3a1_wpct,oc_usda_calc_wpct,fe_ox_usda_4g2_wpct,n_tot_usda_4h2_wpct,ph_h2o_usda_4c1_index,ph_cacl2_usda_4c1_index,...,p_meh3_usda_4d6_mgkg,k_meh3_usda_4d6_mgkg,na_meh3_usda_4d6_mgkg,mg_meh3_usda_4d6_mgkg,fe_meh3_usda_4d6_mgkg,pb_meh3_usda_4d6_mgkg,zn_meh3_usda_4d6_mgkg,mo_meh3_usda_4d6_mgkg,si_meh3_usda_4d6_mgkg,sr_meh3_usda_4d6_mgkg
count,64872.000000,2625.000000,25544.000000,64663.000000,45930.000000,64546.000000,25543.000000,64663.000000,48681.000000,48680.000000,...,1511.000000,1511.000000,1511.000000,1511.000000,1511.000000,1511.000000,1511.000000,1511.000000,1511.000000,1511.000000
mean,107448.139891,12.164177,0.261622,5.103172,22.781703,4.731338,0.417243,0.279717,6.425914,5.921011,...,27.728670,187.479071,270.316491,499.479520,133.435287,1.547620,2.260538,0.147248,458.301792,34.589484
std,64279.867075,13.824144,0.494833,10.733414,16.199049,10.786497,0.614882,0.573322,1.317005,1.388367,...,54.330361,199.920511,2562.314643,539.110826,157.129994,2.339409,8.796174,0.197557,272.724600,55.980811
min,123.000000,-10.504993,0.000000,0.000000,-0.040697,-0.936400,0.000000,0.000000,1.970000,2.140000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50359.750000,3.187023,0.064060,0.522988,9.660600,0.340000,0.092084,0.050791,5.390000,4.790000,...,3.511399,78.070083,11.349780,89.443903,53.763637,0.174093,0.311335,0.020105,265.532284,6.269882
50%,96143.500000,7.799616,0.121231,1.433277,20.854123,1.010000,0.239660,0.110000,6.280000,5.720000,...,11.447185,140.691374,28.551094,330.016530,92.926623,0.866050,0.887852,0.090684,440.085186,16.245025
75%,176060.250000,15.993956,0.241587,3.435167,32.359668,2.714600,0.541915,0.233277,7.630000,7.260000,...,30.776656,235.297324,98.591277,752.794747,161.740083,2.480293,2.175833,0.175240,615.688240,33.861683
max,200831.000000,118.798651,8.802493,78.452136,96.144211,78.450000,20.966663,41.901014,10.700000,10.680000,...,821.094690,3050.820000,94743.286930,5608.598342,2708.083047,49.711732,314.719259,1.667415,2517.545244,525.064519


In [38]:
soilsite.describe()
#115K records

,longitude_wgs84_dd,latitude_wgs84_dd,location_error_any_m,id_project_ascii_c,layer_sequence_usda_uint16,layer_upper_depth_usda_cm,layer_lower_depth_usda_cm,horizon_designation_discontinuity_usda_c
count,115360.000000,115367.000000,108548.000000,117218.000000,117218.000000,116675.000000,113603.000000,4775.000000
mean,-99.694761,41.490550,18447.971174,4407.714404,107.773542,25.860383,44.037022,2.207958
std,19.749183,7.045919,17329.315168,974.315386,129.365506,43.742321,53.504713,0.543171
min,-167.994145,-77.502194,30.000000,4.000000,1.000000,0.000000,0.000000,2.000000
25%,-112.700730,38.460556,30.000000,4262.000000,21.000000,0.000000,10.000000,2.000000
50%,-97.718955,42.253505,20708.270621,4662.000000,53.000000,7.000000,24.000000,2.000000
75%,-85.443165,46.109565,30541.864690,4913.000000,140.000000,36.000000,63.000000,2.000000
max,162.064750,77.573917,122431.660164,5374.000000,657.000000,1737.000000,1783.000000,7.000000


In [61]:
# From Mir sample record is too big to load all
get_sample_record(db,'mir')

{'_id': ObjectId('645bb3def1aab744907e5258'),
 'id_scan_uuid_c': '8e68af029450b1975558673cab936536',
 'id_scan_local_c': '286833',
 'id_layer_uuid_c': '10558c90af851cd4ca4ed34d950ec30e',
 'id_layer_local_c': 195842,
 'model_name_utf8_txt': 'Bruker Vertex 70 with HTS-XT accessory',
 'model_code_any_c': 'Bruker_Vertex_70.HTS.XT',
 'method_light_source_any_c': 'KBr',
 'method_preparation_any_c': '',
 'scan_file_any_c': '286833XS01.0; 286833XS02.0; 286833XS03.0; 286833XS04.0',
 'scan_date_begin_iso_8601_yyyy_mm_dd': '2019-10-18',
 'scan_date_end_iso_8601_yyyy_mm_dd': '2019-10-18',
 'scan_license_title_ascii_txt': 'CC-BY',
 'scan_license_address_idn_url': 'https://ncsslabdatamart.sc.egov.usda.gov/datause.aspx',
 'scan_doi_idf_c': '',
 'scan_contact_name_utf8_txt': 'Scarlett Murphy',
 'scan_contact_email_ietf_email': 'Scarlett.Murphy@usda.gov',
 'scan_mir_nafreq_ossl_pct': 0.0,
 'scan_mir_negfreq_ossl_pct': 0.0,
 'scan_mir_extfreq_ossl_pct': 0.0,
 'scan_mir_600_abs': 1575.0,
 'scan_mir_602_a

In [62]:
# From Mir sample record is too big to load all
get_sample_record(db,'visnir')

{'_id': ObjectId('645bb60af1aab744908206de'),
 'id_scan_uuid_c': '29a3645ebc352c3640ea999d9ca69cd3',
 'id_scan_local_c': 119377,
 'id_layer_uuid_c': 'b71bde8a99f154ed2c35313785acdba1',
 'id_layer_local_c': 46598,
 'model_name_utf8_txt': 'ASD Labspec 2500 with Muglight accessory',
 'model_code_any_c': 'ASD_Labspec_2500_MA',
 'method_light_source_any_c': '',
 'method_preparation_any_c': '',
 'scan_file_any_c': '119377MD01.asd',
 'scan_license_title_ascii_txt': 'CC-BY',
 'scan_license_address_idn_url': 'https://ncsslabdatamart.sc.egov.usda.gov/datause.aspx',
 'scan_doi_idf_c': '',
 'scan_contact_name_utf8_txt': 'Scarlett Murphy',
 'scan_contact_email_ietf_email': 'Scarlett.Murphy@usda.gov',
 'scan_visnir_nafreq_ossl_pct': 0.0,
 'scan_visnir_negfreq_ossl_pct': 0.0,
 'scan_visnir_extfreq_ossl_pct': 0.0,
 'scan_visnir_350_pcnt': 5.0,
 'scan_visnir_352_pcnt': 5.0,
 'scan_visnir_354_pcnt': 4.9,
 'scan_visnir_356_pcnt': 4.5,
 'scan_visnir_358_pcnt': 4.5,
 'scan_visnir_360_pcnt': 4.4,
 'scan_vis

In [50]:
get_collection_fields(db,'soillab')

dict_keys(['_id', 'id_layer_uuid_c', 'id_layer_local_c', 'sample_contact_name_utf8_txt', 'sample_contact_email_ietf_email', 'acid_tea_usda4b2_cmolkg', 'al_ox_usda_4g2_wpct', 'c_tot_usda_4h2_wpct', 'clay_tot_usda_3a1_wpct', 'oc_usda_calc_wpct', 'fe_ox_usda_4g2_wpct', 'n_tot_usda_4h2_wpct', 'ph_h2o_usda_4c1_index', 'ph_cacl2_usda_4c1_index', 'ph_naf_usda_4c1_index', 'sand_tot_usda_3a1_wpct', 'silt_tot_usda_3a1_wpct', 'clay_tot_usda_3a1_wpct_1', 's_tot_usda_4h2_wpct', 'wr_1500kbar_usda_3c2_wpct'])

In [63]:
get_sample_record(db,'soillab')

{'_id': ObjectId('645bb550f1aab744907fc48a'),
 'id_layer_uuid_c': '480f41ca00f52e4d830b60438ce41685',
 'id_layer_local_c': 197861,
 'sample_contact_name_utf8_txt': 'Scarlett Murphy',
 'sample_contact_email_ietf_email': 'Scarlett.Murphy@usda.gov',
 'bsat_usda_4b4_wpct': 94.0,
 'bd_clod_usda_3b1_gcm3': 1.28869193,
 'bd_od_usda_3b2_gcm3': 1.57643479,
 'ca_ext_usda_4b1_cmolkg': 11.1547593,
 'c_tot_usda_4h2_wpct': 0.33808661,
 'cec_ext_usda_4b1_cmolkg': 14.3999532,
 'clay_tot_usda_3a1_wpct': 18.3806339,
 'ec_w_usda_4f1_dsm': 0.09505703,
 'oc_usda_calc_wpct': 0.34,
 'mg_ext_usda_4b1_cmolkg': 1.8773946,
 'n_tot_usda_4h2_wpct': 0.10245049,
 'ph_h2o_usda_4c1_index': 6.12,
 'ph_cacl2_usda_4c1_index': 5.4,
 'p_ext_usda_4d6_mgkg': 16.3074199,
 'k_ext_usda_4b1_cmolkg': 0.3500309,
 'sand_tot_usda_3a1_wpct': 16.0,
 'silt_tot_usda_3a1_wpct': 65.6,
 'clay_tot_usda_3a1_wpct_1': 18.3806339,
 'na_ext_usda_4b1_cmolkg': 0.0396874,
 's_tot_usda_4h2_wpct': 0.01639208,
 'sum_bases_4b4b2a_cmolkg': 13.5,
 'wr_33

In [60]:
#id_layer_uuid_c connects the data in the collections
display(get_data_collection(db,'mir',query={'id_layer_uuid_c': 'a0f56bb43799522951d7a3d3a5b4e597'}))
display(get_data_collection(db,'soillab',query={'id_layer_uuid_c': 'a0f56bb43799522951d7a3d3a5b4e597'}))
display(get_data_collection(db,'soilsite',query={'id_layer_uuid_c': 'a0f56bb43799522951d7a3d3a5b4e597'}))

,_id,id_scan_uuid_c,id_scan_local_c,id_layer_uuid_c,id_layer_local_c,model_name_utf8_txt,model_code_any_c,method_light_source_any_c,method_preparation_any_c,scan_file_any_c,...,scan_mir_3984_abs,scan_mir_3986_abs,scan_mir_3988_abs,scan_mir_3990_abs,scan_mir_3992_abs,scan_mir_3994_abs,scan_mir_3996_abs,scan_mir_3998_abs,scan_mir_4000_abs,_row
0,645bb31ef1aab744907df3e8,2469a7006a4ee9b5559f754578f21049,139832,a0f56bb43799522951d7a3d3a5b4e597,56286,Bruker Vertex 70 with HTS-XT accessory,Bruker_Vertex_70.HTS.XT,KBr,,139832XS01.0; 139832XS02.0; 139832XS03.0; 1398...,...,148.0,148.0,147.0,147.0,146.0,146.0,146.0,145.0,145.0,2469a7006a4ee9b5559f754578f21049


,_id,id_layer_uuid_c,id_layer_local_c,sample_contact_name_utf8_txt,sample_contact_email_ietf_email,al_dith_usda_4g1_wpct,bsat_usda_4b4_wpct,bd_clod_usda_3b1_gcm3,bd_od_usda_3b2_gcm3,ca_ext_usda_4b1_cmolkg,...,p_ext_usda_4d6_mgkg,k_ext_usda_4b1_cmolkg,sand_tot_usda_3a1_wpct,silt_tot_usda_3a1_wpct,clay_tot_usda_3a1_wpct_1,na_ext_usda_4b1_cmolkg,s_tot_usda_4h2_wpct,sum_bases_4b4b2a_cmolkg,wr_33kbar_usda_3c1_wpct,wr_1500kbar_usda_3c2_wpct
0,645bb546f1aab744907f1a78,a0f56bb43799522951d7a3d3a5b4e597,56286,Scarlett Murphy,Scarlett.Murphy@usda.gov,0.030222,100.0,1.992791,2.024482,39.115208,...,0.200853,0.133651,43.5,41.8,14.654497,0.0,0.0,41.7,10.998674,6.603774


,_id,id_location_olc_c,id_layer_uuid_c,observation_ogc_schema_title_ogc_txt,observation_ogc_schema_idn_url,observation_date_begin_iso_8601_yyyy_mm_dd,observation_date_end_iso_8601_yyyy_mm_dd,location_address_utf8_txt,location_country_iso_3166_c,location_method_any_c,...,id_user_site_ascii_c,pedon_taxa_usda_c,layer_sequence_usda_uint16,layer_type_usda_c,layer_field_label_any_c,layer_upper_depth_usda_cm,layer_lower_depth_usda_cm,horizon_designation_usda_c,horizon_designation_discontinuity_usda_c,layer_texture_usda_c
0,645bb558f1aab74490801cdd,86HQF26F+M9,a0f56bb43799522951d7a3d3a5b4e597,Open Soil Spectroscopy Library,https://soilspectroscopy.github.io,2010-11-25,2010-11-25,De Kalb Co.,USA,GPS,...,S10IN033002,"Fine-loamy, illitic, mesic Oxyaquic Argiudoll",11,,,94.0,115.0,B__2,2,


# Q1: What Predicts Carbon? (Work in progress)

Columns:

- oc_iso.10694_w.pct	Organic Carbon,	weight percent, Estimated Organic Carbon based on Total C
- oc_usda.c1059_w.pct	Organic Carbon, Total C without CaCO3, 	weight percent,	Estimated Organic Carbon based on Total C
- oc_usda.c729_w.pct	Organic Carbon, Total C without CaCO3, S prep	weight percent

In [ ]:
mir.describe()

In [ ]:
visnir.describe()

In [8]:
test[:10].to_csv(f'{data_dir}/small.csv')

In [12]:
test.describe()

,longitude.point_wgs84_dd,latitude.point_wgs84_dd,layer.sequence_usda_uint16,layer.upper.depth_usda_cm,layer.lower.depth_usda_cm,longitude.county_wgs84_dd,latitude.county_wgs84_dd,location.point.error_any_m,acidity_usda.a795_cmolc.kg,aggstb_usda.a1_w.pct,...,scan_visnir.2482_ref,scan_visnir.2484_ref,scan_visnir.2486_ref,scan_visnir.2488_ref,scan_visnir.2490_ref,scan_visnir.2492_ref,scan_visnir.2494_ref,scan_visnir.2496_ref,scan_visnir.2498_ref,scan_visnir.2500_ref
count,87707.000000,87707.000000,7455.000000,134827.000000,132148.000000,83222.000000,83222.000000,87707.000000,30061.000000,3218.000000,...,64644.000000,64644.000000,64644.000000,64644.000000,64644.000000,64644.000000,64644.000000,64644.000000,64644.000000,64644.000000
mean,-38.610303,41.898495,2.771831,23.797749,45.729387,-98.936793,40.705190,484.097723,14.649964,37.037290,...,0.368747,0.368093,0.367478,0.366915,0.366415,0.365985,0.365614,0.365312,0.365081,0.364867
std,60.729395,15.616700,2.305919,42.129258,50.385662,17.716686,6.497736,484.162796,24.740795,30.384008,...,0.087191,0.087128,0.087068,0.087019,0.086977,0.086943,0.086917,0.086894,0.086883,0.086888
min,-173.198001,-46.420000,0.000000,0.000000,0.000000,-167.994145,17.980180,30.000000,0.000000,0.000000,...,0.071247,0.071217,0.070881,0.071156,0.070903,0.070679,0.070936,0.071058,0.071069,0.070392
25%,-97.112503,38.838974,1.000000,0.000000,20.000000,-111.549030,37.191760,30.000000,3.499710,9.000000,...,0.310723,0.310050,0.309408,0.308865,0.308392,0.307928,0.307547,0.307253,0.307030,0.306842
50%,-5.155099,43.535632,2.000000,2.000000,20.000000,-96.688800,40.733550,30.000000,7.072200,28.000000,...,0.366867,0.366178,0.365537,0.364953,0.364406,0.363936,0.363559,0.363240,0.362983,0.362768
75%,14.421631,48.915910,4.000000,33.000000,61.000000,-86.260570,44.545095,1000.000000,13.802810,63.000000,...,0.423077,0.422329,0.421630,0.420994,0.420454,0.419968,0.419590,0.419228,0.419006,0.418781
max,177.910000,71.316556,56.000000,1676.000000,1692.000000,0.315575,69.694275,1111.000000,282.720020,106.000000,...,0.877398,0.875266,0.872904,0.870909,0.869880,0.870378,0.872813,0.877563,0.885003,0.895508


In [15]:
test.columns

Index(['dataset.code_ascii_txt', 'id.layer_local_c', 'id.layer_uuid_txt',
       'id.project_ascii_txt', 'id.location_olc_txt',
       'id.dataset.site_ascii_txt', 'id.scan_local_c',
       'longitude.point_wgs84_dd', 'latitude.point_wgs84_dd',
       'layer.sequence_usda_uint16',
       ...
       'scan_visnir.2482_ref', 'scan_visnir.2484_ref', 'scan_visnir.2486_ref',
       'scan_visnir.2488_ref', 'scan_visnir.2490_ref', 'scan_visnir.2492_ref',
       'scan_visnir.2494_ref', 'scan_visnir.2496_ref', 'scan_visnir.2498_ref',
       'scan_visnir.2500_ref'],
      dtype='object', length=2910)